In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 

#path = os.path.join(os.getcwd(), "plots")

not here


In [2]:
grid_data_file = os.path.join(os.getcwd().split('data')[0], 'bi.pickle')
with open(grid_data_file,'rb') as f:
    load_array= pickle.load(f)
exc = [load_array[0][20], load_array[0][50]]
inh = [load_array[1][20], load_array[1][50]]

folder = str(os.getcwd().split('data_2')[1])
print(folder)
case = str(folder[1])

file_ = 'down.pickle'

c_var = [0,1]
if len(folder) == 4:
    if folder[3] == 'e':
        p_var = [0]
    elif folder[3] == 'i':
        p_var = [1]
else:
    p_var = [0,1]
    
print(p_var)

step1 = 100
step2 = 10
step3 = 1

/2_ei
[0, 1]


In [3]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

In [4]:
d_array = np.arange(500., 20, -0.1)
t0_array = d_array - 20.
t_pen = np.zeros(( t0_array.shape ))
for i in range(len(t_pen)):
    t_pen[i] = t0_array[i] / d_array[i]

In [13]:
dur_pre = 10
dur_post = 10

n_pre = int(np.around(dur_pre/aln.params.dt + 1.,1))
n_post = int(np.around(dur_post/aln.params.dt + 1.,1))

tol = 1e-8
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

if case in ['1', '2']:    # low to high
    max_I = [3., -3.]
else:
    max_I = [-3., 3.]
    
if case in ['1', '3']:    # sparsity
    factor_ws = 1.
    factor_we = 0.
    cost_index = 2
else:
    factor_ws = 0.
    factor_we = 1.
    cost_index = 1
    
maxC = [1000., -1000., 0.18, 0.]
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

In [6]:
bestControl_ = np.array( [[None] * len(t_pen)] * len(exc) )
bestState_ = np.array( [[None] * len(t_pen)] * len(exc) )
cost_ = np.array( [[None] * len(t_pen)] * len(exc) )
costnode_ = np.array( [[None] * len(t_pen)] * len(exc) )
weights_ = np.array( [[None] * len(t_pen)] * len(exc) )
convergence_ = np.array( [[None] * len(t_pen)] * len(exc) )

initVars = [None] * len(exc)
target = [None] * len(exc)
cost_uncontrolled = [None] * len(exc)

print(bestControl_.shape)

(2, 4800)


In [7]:
if os.path.isfile(file_) :
    print("file found")
    
    with open(file_,'rb') as f:
        load_array = pickle.load(f)

    bestControl_ = load_array[0]
    bestState_ = load_array[1]
    cost_ = load_array[2]
    costnode_ = load_array[3]
    weights_ = load_array[4]

file found


In [8]:
# get initial parameters and target states

i_range = range(len(exc))
data.set_parameters(aln)
max_percent = []

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    aln.params.duration = 3000.
    
    control0 = aln.getZeroControl()
    control0 = functions.step_control(aln, maxI_ = max_I[0])

    aln.run(control=control0)
    
    target_rates = np.zeros((2))
    target_rates[0] = aln.rates_exc[0,-1] 
    target_rates[1] = aln.rates_inh[0,-1]

    control0 = functions.step_control(aln, maxI_ = max_I[1])
    aln.run(control=control0)

    init_state_vars = np.zeros(( len(state_vars) ))
    for j in range(len(state_vars)):
        if aln.state[state_vars[j]].size == 1:
            init_state_vars[j] = aln.state[state_vars[j]][0]
        else:
            init_state_vars[j] = aln.state[state_vars[j]][0,-1]

    initVars[i] = init_state_vars
    target[i] = [target_rates[0], target_rates[1]]
    
    max_percent.append( (np.argmax(np.abs(bestControl_[i][0]), axis=2)[0,1] -100) / 4800 )

-------  0 0.4500000000000001 0.4750000000000002
-------  1 0.47500000000000014 0.6000000000000003


In [19]:
# get uncontrolled cost

data.set_parameters(aln)

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    dur = d_array[0]
    aln.params.duration = dur
    
    target_ = aln.getZeroTarget()
    target_[:,0,:] = target[i][0]
    target_[:,1,:] = target[i][1]
            
    cost.setParams(1.0, 0.0, 0.0)

    setinit(initVars[i], aln)
    control0 = aln.getZeroControl()

    # "HS", "FR", "PR", "HZ"
    cgv = None
    max_it = 0

    bestControl_init_, bestState_init_, cost_init_, runtime_init_, grad_init_, phi_init_, phi1_, costnode_init_ = aln.A1(
        control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
        prec_variables_ = p_var, transition_time_ = t_pen[0])
    
    cost_uncontrolled[i] = sum(costnode_init_[0][0][:])
    print("uncontrolled cost = ", cost_uncontrolled[i])

-------  0 0.4500000000000001 0.4750000000000002
set cost params:  1.0 0.0 0.0
interpolate adjoint :  True True True
initial cost =  1468888.8173558703
RUN  0 , total integrated cost =  1468888.8173558703
Gradient descend method:  None
RUN  0 , total integrated cost =  1468888.8173558703
Improved over  0  iterations in  0.0  seconds by  0.0  percent.
uncontrolled cost =  1468888.8173558684
-------  1 0.47500000000000014 0.6000000000000003
set cost params:  1.0 0.0 0.0
interpolate adjoint :  True True True
initial cost =  1475251.059202799
RUN  0 , total integrated cost =  1475251.059202799
Gradient descend method:  None
RUN  0 , total integrated cost =  1475251.059202799
Improved over  0  iterations in  0.0  seconds by  0.0  percent.
uncontrolled cost =  1475251.0592028003


In [ ]:
for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    j = 0
    step = step1

    while j < 4799:

        if j == 0:
            j += step
            continue
            
        if not convergence_[i][j-step]:
            if j - step > 0:
                j -= step
                print('go back to -----', j)

        print('-----', j)

        if convergence_[i][j]:
            j += step
            continue

        dur = d_array[j]
        T = 5001 - j
        aln.params.duration = dur

        cost.setParams(1.0, factor_we, factor_ws)
        weights_[i][j] = cost.getParams()

        setinit(initVars[i], aln)

        max_it = 10000

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        if type(bestControl_[i][j]) == type(None):
            max_percent_prev = (np.argmax(np.abs(bestControl_[i][j-step]), axis=2)[0,1] - 100) / (T-200)
            shift_ind = 0
            if j < 3000:
                while max_percent_prev > max_percent[i]:
                    shift_ind += 1
                    max_percent_prev = (np.argmax(np.abs(bestControl_[i][j-step]), axis=2)[0,1] - 100 - shift_ind) / (T-200)

            control0 = np.zeros(( bestControl_[i][j-step][:,:,100:-(100+step)].shape ))
            print('shift ind = ', shift_ind)
            if shift_ind != 0:
                control0[:,:,:-shift_ind] = 1. * bestControl_[i][j-step][:,:,100+shift_ind:-(100+step)]
            else:
                control0[:,:,:] = 1. * bestControl_[i][j-step][:,:,100:-(100+step)]
                #control0[0,0,:] = 0.
        else:
            if bestControl_[i][j].shape[2] == T+200:
                control0 = bestControl_[i][j][:,:,100:-100]
                #control0[0,1,:] = 0.
            else:
                control0 = bestControl_[i][j][:,:,100:-(100+j)]

        bestControl_[i][j], bestState_[i][j], cost_[i][j], runtime_, grad_, phi_, phi1_, costnode_[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])
        
        time_ = np.arange(-dur_pre,round(dur+dur_post+aln.params.dt,1),aln.params.dt)
        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        print(costnode_[i][j][0][0][0], costnode_[i][j][cost_index][0][:2])

        if cost_[i][j][-1] == 0.:
            convergence_[i][j] = True
            if j >= 4400:
                step = step2
            if j >=4790:
                step = step3

        with open(file_,'wb') as f:
            pickle.dump([bestControl_, bestState_, cost_, costnode_, weights_], f)

        j += step

-------  0 0.4500000000000001 0.4750000000000002
----- 100
set cost params:  1.0 0.0 1.0
interpolate adjoint :  True True True
initial cost =  3.3311692425257844
RUN  0 , total integrated cost =  3.3311692425257844
Gradient descend method:  None
RUN  1 , total integrated cost =  3.3311692425257844
Control only changes marginally.
RUN  1 , total integrated cost =  3.3311692425257844
Improved over  1  iterations in  10.251213957555592  seconds by  0.0  percent.
3.361728244356625e-10 [7.53690786e-36 5.54834423e+00]
----- 200
set cost params:  1.0 0.0 1.0
interpolate adjoint :  True True True
initial cost =  3.3311692694247457
RUN  0 , total integrated cost =  3.3311692694247457
Gradient descend method:  None
RUN  1 , total integrated cost =  3.3311692694247457
Control only changes marginally.
RUN  1 , total integrated cost =  3.3311692694247457
Improved over  1  iterations in  0.2710532722994685  seconds by  0.0  percent.
5.346061399900658e-10 [7.53689509e-36 5.54834431e+00]
----- 300
set

In [10]:
bestControl_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
bestState_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
cost_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
costnode_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
weights_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
convergence_maxW = np.array( [[None] * len(t_pen)] * len(exc) )

file_maxW = 'down_Wmax.pickle'

In [11]:
if os.path.isfile(file_maxW) :
    print("file found")
    
    with open(file_maxW,'rb') as f:
        load_array = pickle.load(f)

    bestControl_maxW = load_array[0]
    bestState_maxW = load_array[1]
    cost_maxW = load_array[2]
    costnode_maxW = load_array[3]
    weights_maxW = load_array[4]

file found


In [14]:
for i in i_range:
    print('------------', i)
    j = 0

    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.

    aln.params.duration = dur

    while not convergence_maxW[i][j]:
        dur = d_array[j]
        aln.params.duration = dur

        setinit(initVars[i], aln)

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        control0 = aln.getZeroControl()
        if type(bestControl_maxW[i][j]) == type(None):
            max_it = 10
            control0 = bestControl_[i][0][:,:,n_pre-1:-n_post+1]
            weight_ = ( cost_uncontrolled[i] * (1. - t_pen[0]) / (1. - t_pen[j]) 
                       - sum(costnode_[i][j][0][0][:]) ) / sum( costnode_[i][j][cost_index][0][:] )
            cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()
        else:
            control0 = bestControl_maxW[i][j][:,:,n_pre-1:-n_post+1]
            weight_ = ( cost_uncontrolled[i] * (1. - t_pen[0]) / (1. - t_pen[j])
                       - sum(costnode_maxW[i][j][0][0][:]) ) / sum( costnode_maxW[i][j][cost_index][0][:] )
            cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()
            max_it = int(100)

        bestControl_maxW[i][j], bestState_maxW[i][j], cost_maxW[i][j], runtime_, grad_, phi_, phi1_, costnode_maxW[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])

        if cost_maxW[i][j][2] == 0.:
            convergence_maxW[i][j] = True

        print('cost splitting : ', costnode_maxW[i][j][0][0][0], costnode_maxW[i][j][cost_index][0][:2])

        with open(file_maxW,'wb') as f:
            pickle.dump([bestControl_maxW, bestState_maxW, cost_maxW, costnode_maxW, weights_maxW], f)

------------ 0
set cost params:  1.0 413464.46905726823 0.0
interpolate adjoint :  True True True
initial cost =  1468888.8173558656
RUN  0 , total integrated cost =  1468888.8173558656
Gradient descend method:  None
RUN  1 , total integrated cost =  1468888.8173558656
Control only changes marginally.
RUN  1 , total integrated cost =  1468888.8173558656
Improved over  1  iterations in  1.1489928580122069  seconds by  0.0  percent.
cost splitting :  2.561811036912724e-07 [1.38096713 2.17166902]
------------ 1
set cost params:  1.0 135336.17854099625 0.0
interpolate adjoint :  True True True
initial cost =  1475251.0592027896
RUN  0 , total integrated cost =  1475251.0592027896
Gradient descend method:  None
RUN  1 , total integrated cost =  1475251.0592027896
Control only changes marginally.
RUN  1 , total integrated cost =  1475251.0592027896
Improved over  1  iterations in  1.134532677999232  seconds by  0.0  percent.
cost splitting :  1.6447899034812753e-09 [6.94580352 3.9548372 ]


In [70]:
for j in np.arange(4770,4800,1):
    bestControl_maxW[1][j] = None
    convergence_maxW[1][j] = None

In [71]:
for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    j = 4770
    step = step1
    
    if i == 0:
        continue

    while j < 4800:

        if j == 0:
            j += step
            continue
            
        if j >= 4400:
            step = step2
        if j >4790:
            step = step3
            
        if not convergence_maxW[i][j-step]:
            if j - step > 0:
                j -= step
                print('go back to -----', j)

        print('-----', j)

        if convergence_maxW[i][j]:
            j += step
            continue

        dur = d_array[j]
        T = 5001 - j
        aln.params.duration = dur
        setinit(initVars[i], aln)

        max_it = 10

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        if type(bestControl_maxW[i][j]) == type(None):
            max_percent_prev = (np.argmax(np.abs(bestControl_maxW[i][j-step]), axis=2)[0,1] - 100) / (T-200)
            shift_ind = 0
            if j < 4000:
                while max_percent_prev > max_percent[i]:
                    shift_ind += 1
                    max_percent_prev = (np.argmax(np.abs(bestControl_maxW[i][j-step]), axis=2)[0,1] - 100 - shift_ind) / (T-200)

            control0 = np.zeros(( bestControl_maxW[i][j-step][:,:,100:-(100+step)].shape ))
            print('shift ind = ', shift_ind)
            if shift_ind != 0:
                control0[:,:,:-shift_ind] = 1. * bestControl_maxW[i][j-step][:,:,100+shift_ind:-(100+step)]
            else:
                control0[:,:,:] = 1. * bestControl_maxW[i][j-step][:,:,100:-(100+step)]
                
            #print(cost_uncontrolled[i] * (1. - t_pen[0]) / (1. - t_pen[j]),  sum(costnode_maxW[i][j-step][0][0][:]) , sum( costnode_maxW[i][j-step][cost_index][0][:]) )
                
            weight_ = ( cost_uncontrolled[i] * (1. - t_pen[0]) / (1. - t_pen[j])
                          - sum(costnode_maxW[i][j-step][0][0][:]) ) / sum( costnode_maxW[i][j-step][cost_index][0][:] )
            cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()
            max_it = 1
        
        else:
            if bestControl_maxW[i][j].shape[2] == T+200:
                control0 = bestControl_maxW[i][j][:,:,100:-100]
                #control0[0,1,:] = 0.
            else:
                control0 = bestControl_maxW[i][j][:,:,100:-(100+j)]
                
            weight_ = ( cost_uncontrolled[i] * (1. - t_pen[0]) / (1. - t_pen[j])
                       - sum(costnode_maxW[i][j][0][0][:]) ) / sum( costnode_maxW[i][j][cost_index][0][:] )
            cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()
            
        if j >= 4770:
            cost.setParams(1., 800. * factor_we, 800. * factor_ws)
            weights_maxW[i][j] = cost.getParams()

        bestControl_maxW[i][j], bestState_maxW[i][j], cost_maxW[i][j], runtime_, grad_, phi_, phi1_, costnode_maxW[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])

        print(costnode_maxW[i][j][0][0][:2], costnode_maxW[i][j][cost_index][0][:2])

        if max_it > 1:
            if cost_maxW[i][j][2] == 0.:
                convergence_maxW[i][j] = True
                if j >= 4400:
                    step = step2
                if j >=4790:
                    step = step3

        with open(file_maxW,'wb') as f:
            pickle.dump([bestControl_maxW, bestState_maxW, cost_maxW, costnode_maxW, weights_maxW], f)

        j += step

-------  0 0.4500000000000001 0.4750000000000002
-------  1 0.47500000000000014 0.6000000000000003
----- 4770
shift ind =  0
set cost params:  1.0 1507.034477342776 0.0
set cost params:  1.0 800.0 0.0
interpolate adjoint :  True True True
initial cost =  50757.8854140288
RUN  0 , total integrated cost =  50757.8854140288
Gradient descend method:  None
RUN  1 , total integrated cost =  48879.79587222067
RUN  1 , total integrated cost =  48879.79587222067
Improved over  1  iterations in  0.17901175696169958  seconds by  3.7000941360907404  percent.
[ 7320.00703765 12285.10812831] [27.83198589  8.76136499]
go back to ----- 4770
----- 4770
set cost params:  1.0 1318.7213631121895 0.0
set cost params:  1.0 800.0 0.0
interpolate adjoint :  True True True
initial cost =  48879.79587222067
RUN  0 , total integrated cost =  48879.79587222067
Gradient descend method:  None
RUN  1 , total integrated cost =  47902.914537861594
RUN  2 , total integrated cost =  47091.078519434726
RUN  3 , total int

RUN  7 , total integrated cost =  47644.981573639685
RUN  8 , total integrated cost =  47643.61932307421
RUN  9 , total integrated cost =  47642.739912728124
RUN  10 , total integrated cost =  47641.185720304726
RUN  10 , total integrated cost =  47641.185720304726
Improved over  10  iterations in  1.0897366289864294  seconds by  0.29514471213940396  percent.
[7750.3667466  6415.35429819] [35.35061317  6.49371767]
go back to ----- 4780
----- 4780
set cost params:  1.0 1212.7168611805316 0.0
set cost params:  1.0 800.0 0.0
interpolate adjoint :  True True True
initial cost =  47641.185720304726
RUN  0 , total integrated cost =  47641.185720304726
Gradient descend method:  None
RUN  1 , total integrated cost =  47640.52696071894
RUN  2 , total integrated cost =  47640.09805163722
RUN  3 , total integrated cost =  47639.95686602562
RUN  4 , total integrated cost =  47639.884877797005
RUN  5 , total integrated cost =  47639.714355138334
RUN  6 , total integrated cost =  47639.478874704306
